Make sure to set the runtime to GPU: Runtime -> Change runtime type -> T4 GPU

You can upload your own images, then change the relevant code cells to load it and send it through the model.

In [ ]:
# install the intrinsic decomposition repo from github
!pip install https://github.com/compphoto/Intrinsic/archive/main.zip

In [1]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, uninvert
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_pipeline
from intrinsic.model_util import load_models

In [2]:
# download the pretrained weights and return the model (may take a bit to download weights)
intrinsic_model = load_models('paper_weights')

Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v1.0/final_weights.pt" to /tmp/xdg-cache/torch/hub/checkpoints/final_weights.pt
100%|██████████| 485M/485M [00:07<00:00, 70.5MB/s] 
Downloading: "https://github.com/facebookresearch/WSL-Images/zipball/main" to /tmp/xdg-cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /tmp/xdg-cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth
100%|██████████| 340M/340M [00:01<00:00, 343MB/s] 
/opt/conda/lib/python3.9/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use loa

In [ ]:
# load an example image from the github repo
torch.hub.download_url_to_file('https://raw.githubusercontent.com/compphoto/Intrinsic/main/figures/avocado.png', 'avo.png')

In [ ]:
# load the image to run through the pipeline
img = load_image('/content/avo.png')

In [ ]:
# run the image through the pipeline (use R0 resizing dicussed in the paper)
result = run_pipeline(
    intrinsic_model,
    img,
    resize_conf=0.0,
    maintain_size=True,
    linear=False,
    device='cuda'
)

In [ ]:
# convert the inverse shading to regular shading for visualization
shd = uninvert(result['inv_shading'])
alb = result['albedo']

In [ ]:
# show the result (gamma corrects the linear intrinsic components and scales to [0-1])
show([img, view(shd), view(alb)], size=(20, 7))

# Ester - apply Intrinsic to our project images

In [10]:
import os
from os import listdir
import matplotlib.pyplot as plt
 
input_folder = "../scene-representation-diffusion-model/datasets/images/"
output_folder = "../scene-representation-diffusion-model/datasets/shading/"
for input_image in os.listdir(input_folder):
 
    # check if the image ends with png
    if (input_image.endswith(".png")):
        img = load_image(input_folder + input_image)[:,:,:3]
        
        # run the image through the pipeline (use R0 resizing dicussed in the paper)
        result = run_pipeline(
            intrinsic_model,
            img,
            resize_conf=0.0,
            maintain_size=True,
            linear=False,
            device='cuda'
        )
        
        # convert the inverse shading to regular shading for visualization
        shd = uninvert(result['inv_shading'])
        
        # save the shading image in a folder 
        plt.imsave(output_folder + input_image, view(shd), cmap='gray')
        